# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
#Get Data
data = "city_weather.csv"
df = pd.read_csv(data)
df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,wind
0,0,leningradskiy,98.0,RU,1.580009e+09,96.0,69.38,178.42,-12.55,4.12
1,1,ilulissat,92.0,GL,1.580009e+09,46.0,69.22,-51.10,6.80,10.29
2,2,ushuaia,75.0,AR,1.580009e+09,75.0,-54.80,-68.30,46.40,23.04
3,3,new norfolk,40.0,AU,1.580009e+09,35.0,-42.78,147.06,71.60,18.34
4,4,castro,0.0,BR,1.580009e+09,93.0,-24.79,-50.01,61.63,2.84


In [18]:
#Configure map
gmaps.configure(api_key=g_key)

#Get Locations
coordinates = df[["Latitude", "Longitude"]].dropna().astype(float)

#Get humidity for weight
humidity = df["Humidity"].dropna().astype(float)

In [19]:
#Create map
fig = gmaps.figure()

#Create layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

#Add layer
fig.add_layer(heat_layer)

#show map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#Narrow down the search
df.dropna()
df = df.loc[df["Max Temp"] > 80]
df = df.loc[df["Max Temp"] < 90]
df = df.loc[df["Cloudiness"] < 50]
df = df.loc[df["Humidity"] < 80]
df = df.loc[df["wind"] < 20]
df = df.reset_index()
df.head()

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,wind
0,11,11,tiarei,40.0,PF,1.580009e+09,70.0,-17.53,-149.33,86.00,8.05
1,45,45,kefamenanu,33.0,ID,1.580009e+09,62.0,-9.45,124.48,87.37,2.57
2,50,50,geraldton,15.0,AU,1.580009e+09,58.0,-28.77,114.60,82.40,12.75
3,95,95,santa cruz,2.0,BO,1.580009e+09,69.0,-17.80,-63.17,80.60,6.93
4,102,102,makakilo city,20.0,US,1.580009e+09,65.0,21.35,-158.09,82.40,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
#Create Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create parameters for search
params = { 
    "rankby": "prominence",
    "radius": "5000",
    "types": "lodging",
    "key": g_key,
}

#use iterrows to iterate through pandas dataframe
for index, row in df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]

    coordinates = (f"{lat},{lng}")

    params['location'] = coordinates
    
    print(f"Retrieving Results for Index {index}: {coordinates}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        
        df.loc[index, 'Hotel Name'] = results[0]['name']
        df.loc[index, 'Hotel Latitude'] = results[0]["geometry"]["location"]["lat"]
        df.loc[index, 'Hotel Longitude'] = results[0]["geometry"]["location"]["lng"]
              
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
print("------------")              

Retrieving Results for Index 0: -17.53,-149.33.
Closest tiarei hotel is Le Rocher de Tahiti.
Retrieving Results for Index 1: -9.45,124.48.
Closest kefamenanu hotel is PAULUS TAITOH.
Retrieving Results for Index 2: -28.77,114.6.
Closest geraldton hotel is Broadwater Mariner Resort.
Retrieving Results for Index 3: -17.8,-63.17.
Closest santa cruz hotel is Hotel Cortez.
Retrieving Results for Index 4: 21.35,-158.09.
Closest makakilo city hotel is Marriott's Ko Olina Beach Club.
Retrieving Results for Index 5: -35.12,139.27.
Closest murray bridge hotel is Balcony On Sixth Lodge.
Retrieving Results for Index 6: -37.25,174.75.
Closest waiuku hotel is Waiuku Lodge Motel.
Retrieving Results for Index 7: 5.41,100.34.
Closest georgetown hotel is Cititel Penang.
Retrieving Results for Index 8: 12.58,-81.7.
Closest san andres hotel is Hotel Arena Blanca.
Retrieving Results for Index 9: -34.65,-58.62.
Closest moron hotel is Hotel Morón.
Retrieving Results for Index 10: -8.84,13.23.
Closest luanda h

In [30]:
 # Create hotel symbol layer
coordinates = df.dropna()
hotels = df['Hotel Name'].tolist()

coordinates = df[["Hotel Latitude", "Hotel Longitude"]]                    
coordinates = coordinates.dropna()                       

In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Hotel Latitude", "Hotel Longitude"]]

In [35]:
#Create Layer
hotel_layer = gmaps.symbol_layer(
    coordinates, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2
)

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display Map
fig



Figure(layout=FigureLayout(height='420px'))